<img style="float: left; margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="300" height="500" /> 
    
    
### <font color='navy'> Simulación de procesos financieros. 

**Nombres:** Diego Pintor Ochoa, Lyha Espinosa García

**Fecha:** 28 de abril de 2021.

**Expediente** : if72622, .
**Profesor:** Oscar David Jaramillo Zuluaga.

# Tarea 9: Clase 21

## Enunciado de tarea 
> Replicar el procedimiento anterior para valoración de opciones 'call', pero en este caso para opciones tipo 'put' y compararlo con el valor teórico de la ecuación de Black-Scholes.

### Solución Lyha Espinosa

In [1]:
# Código de solución 
.
.
.
.


SyntaxError: invalid syntax (<ipython-input-1-9d1c0c122db9>, line 2)

### Solución Diego Pintor


In [3]:
#importar los paquetes que se van a usar
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import datetime
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
%matplotlib inline
#algunas opciones para Pandas
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 6)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 78)
pd.set_option('precision', 3)
# Función para descargar precios de cierre ajustados:
def get_adj_closes(tickers, start_date=None, end_date=None):
    # Fecha inicio por defecto (start_date='2010-01-01') y fecha fin por defecto (end_date=today)
    # Descargamos DataFrame con todos los datos
    closes = web.DataReader(name=tickers, data_source='yahoo', start=start_date, end=end_date)
    # Solo necesitamos los precios ajustados en el cierre
    closes = closes['Adj Close']
    # Se ordenan los índices de manera ascendente
    closes.sort_index(inplace=True)
    return closes

In [4]:
aapl = web.YahooOptions('AAPL')
aapl_opt = aapl.get_all_data().reset_index()
aapl_opt = aapl_opt.set_index('Expiry')

In [52]:
def calc_daily_ret(closes):
    return np.log(closes/closes.shift(1)).iloc[1:]
ticker = ['AAPL']
start_date = '2017-01-01'
closes_aapl = get_adj_closes(ticker, start_date)
ret_aapl = calc_daily_ret(closes_aapl)
# Obtenemos el rendimiento simple
Ri = closes_aapl.pct_change(1).iloc[1:]
# Obtenemos su media y desviación estándar de los rendimientos
mu = ret_aapl.mean()[0]
sigma = ret_aapl.std()[0]
mu, sigma

aapl_opt = aapl.get_all_data().reset_index()
K = 135
indice_opt = aapl_opt.loc[(aapl_opt.Type=='put') & (aapl_opt.Strike==K) & (aapl_opt.Expiry=='2022-09-16')]
i_opt= indice_opt.index
opcion_valuar = aapl_opt.loc[i_opt[0]]

In [77]:
# Fechas de simulaciones
from datetime import date
today = pd.to_datetime(date.today())

# Obtener fecha de cierre de la opción a valuar
expiry = opcion_valuar.Expiry
dates = pd.date_range(today, expiry, freq='B')

ndays = len(dates)
dates = pd.date_range(start= Hoy, periods = ndays, freq='B')
# Generando histograma de rendimientos
ren = calc_daily_ret(closes_aapl) 
y, x = np.histogram(ren['AAPL'], bins=50, density=True)

In [79]:
prob = y/np.sum(y)
values = x[1:]


In [80]:
nscen = 10000
# Rendimientos simulados
ret = np.random.choice(values, ndays*nscen, p=prob)
# Fechas
dates = pd.date_range(start=Hoy,periods=ndays)
# Rendimien en Data Frame
sim_ret_hist = pd.DataFrame(ret.reshape((ndays,nscen)),index=dates)
sim_ret_hist
sim_closes_hist = (closes_aapl.iloc[-1,0])*np.exp(sim_ret_hist.cumsum())


In [81]:
K = opcion_valuar['JSON']['strike']
ndays = len(dates)
nscen = 100000


# Tasa de bonos de 1 yr de fecha 21/04/2021 -> 7%
r = 0.007/360 # Tasa diaria

# Histograma tomando la tasa libre de riesgo
freq, values = np.histogram(ret_aapl+r-mu, bins=2000)
prob = freq/np.sum(freq)

# Simulación de los rendimientos
ret = np.random.choice(values[1:], ndays*nscen, p=prob)


# Simulación de precios
sim_ret_hist = pd.DataFrame(ret.reshape((ndays,nscen)),index=dates)
sim_closes_hist = (closes_aapl.iloc[-1,0]) * np.exp(sim_ret_hist.cumsum())

In [82]:
strike = pd.DataFrame(K*np.ones(ndays*nscen).reshape((ndays,nscen)), index=dates)
call_hist = pd.DataFrame({'Prima':np.exp(-r*ndays) \
                          *np.fmax(strike-sim_closes_hist,np.zeros(ndays*nscen).reshape((ndays,nscen))).T.mean()}, index=dates)


In [83]:
confianza = 0.95
sigma_est = sim_closes_hist.iloc[-1].sem()
mean_est = call_hist.iloc[-1].Prima
i1 = st.t.interval(confianza,nscen-1, loc=mean_est, scale=sigma_est)
i2 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
print(i1)
print(i1)


(14.460483030456672, 15.189275057312912)
(14.460483030456672, 15.189275057312912)


In [84]:
opcion_valuar["JSON"]

{'contractSymbol': 'AAPL220916P00135000',
 'strike': 135.0,
 'currency': 'USD',
 'lastPrice': 19.4,
 'change': 0.0,
 'percentChange': 0.0,
 'volume': 2,
 'openInterest': 1621,
 'bid': 19.5,
 'ask': 19.75,
 'contractSize': 'REGULAR',
 'expiration': 1663286400,
 'lastTradeDate': 1619465019,
 'impliedVolatility': 0.3105232522583008,
 'inTheMoney': True}

In [120]:
call_hist.iloc[-1].values

array([14.82487904])

### Comparando Resultados con Black Scholes

In [117]:
# Datos para usar en la fórmula (anualizados)
vol = ren["AAPL"].std()*np.sqrt(252)
media = ren["AAPL"].mean()*252
maduracion = ndays
st = closes_aapl.iloc[-1,0]
rf = r*360
print("Volatilidad anual",vol)
print("Tiempo de maduración",maduracion,"días")
print("Spot price",st)
print("Strike Price",K)
print("Tasa Libre de riesgo",r*360*100)


Volatilidad anual 0.31638434493720957
Tiempo de maduración 364 días
Spot price 134.7050018310547
Strike Price 135.0
Tasa Libre de riesgo 0.7000000000000001


In [122]:
#Black Scholes
from scipy.stats import norm
d1 = (np.log(st/K)+(rf+(vol**2)/2))/vol
d2 = d1-vol
put = K*np.exp(-rf)*norm.cdf(-d2)-st*norm.cdf(-d1)
print("El valor de la opción con Black Scholes es:",put,"y el valor de la opción simulado es de:",call_hist.iloc[-1].values)

El valor de la opción con Black Scholes es: 16.56963837424412 y el valor de la opción simulado es de: [14.82487904]
